Basic demonstration of the quantum chemistry code I have written. It seems to be fairly quick, but uses quite a bit of ram. One trick which I certainly did not exploit is that some parts of the environments are simply the harmitian conjugate of other parts, and the environments can therefore be compressed further (and should be twice as cheap to calculate).

In [1]:
using Revise,LinearAlgebra, MPSKit, TensorKit, MPSKitExperimental

LinearAlgebra.BLAS.set_num_threads(1);
#TensorKit.TO.disable_cache();
TensorKit.Strided.disable_threads();


FCIDUMP is a well known (though badly documented) format to store the electronic integrals. I have written an experimental parser to load in such files:

In [ ]:
(ERI,K,E0,NORB,NELEC,MS2) = parse_fcidump("N2.CCPVDZ.FCIDUMP")

ERI is the Electronic Repulsion Integral, K is the two-body term, E0 is an energy shift, NORB is the number of orbitals (mps length), NELEC is the number of electrons and MS2 is the total spin.

fcidump files can contain orbital symmetries, but I ignore these.

In [ ]:
(ham,_) = fused_quantum_chemistry_hamiltonian(E0,K,ERI);

The second return value contains some metadata which can be ignored. Instantiate the state:

In [ ]:
psp = Vect[(Irrep[U₁]⊠Irrep[SU₂] ⊠ FermionParity)]((0,0,0)=>1, (1,1//2,1)=>1, (2,0,0)=>1);

left = Vect[(Irrep[U₁]⊠Irrep[SU₂] ⊠ FermionParity)]((-NELEC,MS2//2,mod1(-NELEC,2))=>1);
right = oneunit(left);

virtual = Vect[(Irrep[U₁]⊠Irrep[SU₂] ⊠ FermionParity)]((i,s,b)=>1 for i in -NELEC:0, s in 0:1//2:(MS2//2+1), b in (0,1));

st = FiniteMPS(rand,Float64,NORB,psp,virtual;left,right);

We also need to instantiate the environments, because MPSKit will otherwise call the default environment constructor which keeps everything in ram (I think I even disabled that constructor). To create an environment that lives on disk, we can use:

In [7]:
envs = disk_environments(st,ham);

Let's call DMRG!

In [8]:
@time (st,envs) = find_groundstate(st,ham, DMRG2(trscheme=truncdim(100), verbose = true, maxiter=3),envs);

TaskFailedException: TaskFailedException

    nested task error: TaskFailedException
    Stacktrace:
     [1] wait
       @ ./task.jl:349 [inlined]
     [2] fetch
       @ ./task.jl:369 [inlined]
     [3] (::MPSKitExperimental.var"#181#197"{Dict{Any, MPSKitExperimental.TransposeFact{MPSKitExperimental.DelayedFact{GradedSpace{TensorKit.ProductSector{Tuple{U1Irrep, SU2Irrep, Fermion{TensorKit._fermionparity, Z2Irrep}}}, TensorKit.SortedVectorDict{TensorKit.ProductSector{Tuple{U1Irrep, SU2Irrep, Fermion{TensorKit._fermionparity, Z2Irrep}}}, Int64}}, 3, 2, FusionTree{TensorKit.ProductSector{Tuple{U1Irrep, SU2Irrep, Fermion{TensorKit._fermionparity, Z2Irrep}}}, 3, 1, 2, Nothing}, FusionTree{TensorKit.ProductSector{Tuple{U1Irrep, SU2Irrep, Fermion{TensorKit._fermionparity, Z2Irrep}}}, 2, 0, 1, Nothing}, TensorKit.ProductSector{Tuple{U1Irrep, SU2Irrep, Fermion{TensorKit._fermionparity, Z2Irrep}}}, DataType, TensorMap{GradedSpace{TensorKit.ProductSector{Tuple{U1Irrep, SU2Irrep, Fermion{TensorK

In [9]:
find_groundstate(st,ham,DMRG(),envs)

┌ Info: Iteraton 0 error 0.5819303111535928
└ @ MPSKit /home/maavdamm/.julia/packages/MPSKit/BToBw/src/algorithms/groundstate/dmrg.jl:38


┌ Info: Iteraton 1 error 0.5194024243133698
└ @ MPSKit /home/maavdamm/.julia/packages/MPSKit/BToBw/src/algorithms/groundstate/dmrg.jl:38


┌ Info: Iteraton 2 error 0.06443825400381015
└ @ MPSKit /home/maavdamm/.julia/packages/MPSKit/BToBw/src/algorithms/groundstate/dmrg.jl:38


┌ Info: Iteraton 3 error 0.09377250205710787
└ @ MPSKit /home/maavdamm/.julia/packages/MPSKit/BToBw/src/algorithms/groundstate/dmrg.jl:38


┌ Info: Iteraton 4 error 0.023494130421758606
└ @ MPSKit /home/maavdamm/.julia/packages/MPSKit/BToBw/src/algorithms/groundstate/dmrg.jl:38


...

┌ Info: Iteraton 21 error 0.0013833128306238638
└ @ MPSKit /home/maavdamm/.julia/packages/MPSKit/BToBw/src/algorithms/groundstate/dmrg.jl:38


┌ Info: Iteraton 22 error 0.00082392158291883
└ @ MPSKit /home/maavdamm/.julia/packages/MPSKit/BToBw/src/algorithms/groundstate/dmrg.jl:38


┌ Info: Iteraton 23 error 0.0007686076547961106
└ @ MPSKit /home/maavdamm/.julia/packages/MPSKit/BToBw/src/algorithms/groundstate/dmrg.jl:38


┌ Info: Iteraton 24 error 0.0007542401017896067
└ @ MPSKit /home/maavdamm/.julia/packages/MPSKit/BToBw/src/algorithms/groundstate/dmrg.jl:38


┌ Info: Iteraton 25 error 0.0007382379133939062
└ @ MPSKit /home/maavdamm/.julia/packages/MPSKit/BToBw/src/algorithms/groundstate/dmrg.jl:38


In [18]:
Revise.retry()